In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
start = '2012-01-01'
end = '2024-04-04'
stock = 'AAPL'

data = yf.download(stock, start, end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.433825,302220800
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.500645,260022000
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.639431,271269600
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.771557,318292800
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.751298,394024400


In [4]:
data.reset_index(inplace=True)

In [6]:
ma_100_days=data.Close.rolling(100).mean()

In [7]:
ma_200_days=data.Close.rolling(200).mean()

In [8]:
data.dropna(inplace=True)

In [13]:
int(len(data)*0.80)

2465

In [9]:
data_train = pd.DataFrame(data.Close[0: int(len(data)*0.80)])
data_test = pd.DataFrame(data.Close[int(len(data)*0.80): len(data)])

In [10]:
data_train.shape[0]

2465

In [11]:
data_test.shape[0]

617

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [14]:
data_train_scale = scaler.fit_transform(data_train)

In [18]:
x = []
y = []

for i in range(100, data_train_scale.shape[0]):
    x.append(data_train_scale[i-100:i])
    y.append(data_train_scale[i,0])
print(len(data_train_scale[i-100:i]))

100


In [19]:
x, y = np.array(x), np.array(y)

In [20]:
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [21]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True,
               input_shape = ((x.shape[1],1))))
model.add(Dropout(0.2))

model.add(LSTM(units = 60, activation='relu', return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units =1))

c:\Users\Admin\OneDrive\Desktop\stock_market_analysis\sachinenv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [23]:
model.fit(x,y, epochs = 50, batch_size =32, verbose =1)

Epoch 1/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - loss: 0.0362
Epoch 2/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0049
Epoch 3/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 0.0049
Epoch 4/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 0.0035
Epoch 5/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0033
Epoch 6/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0030
Epoch 7/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0027
Epoch 8/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0031
Epoch 9/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0056
Epoch 10/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0029
Epoch 11/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0036
Epoch 12/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0028
Epoch 13/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0025
Epoch 14/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0025
Epoch 15/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0022
Epoc

In [24]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 50)        │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 60)        │        26,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 60)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100, 80)        │        45,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 80)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 120)            │        96,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 536,285 (2.05 MB)

 Trainable params: 178,761 (698.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 357,524 (1.36 MB)

In [25]:
pas_100_days = data_train.tail(100)

In [26]:
data_test = pd.concat([pas_100_days, data_test], ignore_index=True)

In [27]:
data_test_scale  =  scaler.fit_transform(data_test)

In [28]:
x = []
y = []

for i in range(100, data_test_scale.shape[0]):
    x.append(data_test_scale[i-100:i])
    y.append(data_test_scale[i,0])
x, y = np.array(x), np.array(y)

In [29]:
y_predict = model.predict(x)

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [30]:
print(y_predict)

[[0.24259387]
 [0.24676414]
 [0.2530006 ]
 [0.2614032 ]
 [0.27175978]
 [0.28358018]
 [0.29617974]
 [0.30884376]
 [0.32134324]
 [0.33326262]
 [0.34409076]
 [0.3535404 ]
 [0.36164796]
 [0.36851633]
 [0.37430567]
 [0.37907436]
 [0.38288015]
 [0.38529557]
 [0.38599983]
 [0.38525504]
 [0.38353735]
 [0.38151193]
 [0.38007873]
 [0.38024145]
 [0.38307068]
 [0.38923067]
 [0.39888135]
 [0.4118848 ]
 [0.4268062 ]
 [0.44238824]
 [0.45842588]
 [0.47478342]
 [0.49101555]
 [0.5062084 ]
 [0.5201129 ]
 [0.5336454 ]
 [0.5482099 ]
 [0.5645467 ]
 [0.5833355 ]
 [0.60400164]
 [0.6251473 ]
 [0.6461913 ]
 [0.6653514 ]
 [0.6809744 ]
 [0.6915979 ]
 [0.6975829 ]
 [0.7004448 ]
 [0.70181483]
 [0.7036853 ]
 [0.70703167]
 [0.7121188 ]
 [0.71851945]
 [0.72547543]
 [0.7329996 ]
 [0.74072206]
 [0.74709505]
 [0.75042707]
 [0.7500516 ]
 [0.74605954]
 [0.73967034]
 [0.7323491 ]
 [0.7246489 ]
 [0.7171054 ]
 [0.7094546 ]
 [0.7007362 ]
 [0.6900591 ]
 [0.6768106 ]
 [0.6608861 ]
 [0.64250594]
 [0.6223201 ]
 [0.6011774 ]
 [0.58

In [31]:
scale =1/scaler.scale_

In [32]:
y_predict = y_predict*scale

In [33]:
y = y*scale